
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [59]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

df = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1')

df = df[['OriginalTweet','Sentiment']]
# df['Sentiment'].value_counts()
df = df[df['Sentiment'] != 'Neutral']
df['Sentiment'] = df['Sentiment'].map({'Positive': 1, 'Extremely Positive': 1, 'Negative': 0, 'Extremely Negative': 0})



# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
X = df.iloc[:, 0].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train_test, X_valid, Y_train_test, Y_valid = train_test_split(X, Y, test_size= 0.25001, random_state=42)

X_train, X_test, Y_train, Y_test = train_test_split(X_train_test, Y_train_test, test_size = 0.19997, random_state=42)

# X_train 
# Y_train
# X_test
# Y_test 
# X_valid
# Y_valid

In [92]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.


model = Pipeline(steps=[
("vectorizer", CountVectorizer()),
("lr", LogisticRegression(max_iter=300, penalty='l2', solver='liblinear')),
])


model.fit(X_train, Y_train)

model.predict_proba(["This is amazing"])

array([[0.11010092, 0.88989908]])

In [93]:
from sklearn.metrics import precision_recall_fscore_support

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

Y_pred = model.predict(X_test)

precision, recall, f1, support = precision_recall_fscore_support(Y_test, Y_pred, average=None)

precision_0 = precision[0]
precision_1 = precision[1]
recall_0 = recall[0]
recall_1 = recall[1]

precision_1


np.float64(0.8823529411764706)

In [36]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    pred_prob_1 = model.predict_proba(X)[:, 1]
    pred = 0
    costs = np.zeros(len(Y))

    pred = (pred_prob_1 > threshold).astype(int)

    costs[(Y == 1) & (pred == 0)] = 1
    costs[(Y == 0) & (pred == 1)] = 5

    return costs.mean()



In [95]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.


def get_optim_threshold (model,X,Y, num_thresholds = 101):

    thresholds = np.linspace(0, 1, num_thresholds)
    
    optim_t = None
    optim_cost = np.inf
    
    for t in thresholds:
        c = cost(model, t, X, Y)
        if c < optim_cost:
            optim_cost = c
            optim_t = t
    
    return optim_t, optim_cost




optimal_threshold, cost_at_optimal_threshold = get_optim_threshold(model, X_test, Y_test)

In [109]:

# Part 6


cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

def hoeffding_ci(sample_mean, n, delta=0.01, a=0.0, b=5.0):

    width = b-a
    eps = width * np.sqrt(np.log(2/delta) / (2*n))
    lower_bound = sample_mean - eps
    upper_bound = sample_mean + eps
    return lower_bound, upper_bound


lower_bound, upper_bound = hoeffding_ci(cost_at_optimal_threshold_valid, len(Y_valid), delta=0.01)

cost_interval_valid = (lower_bound, upper_bound)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [108]:

# Part 7


def all_costs(model,threshold,X,Y):
    
    
    pred_prob_1 = model.predict_proba(X)[:, 1]
    pred = 0
    costs = np.zeros(len(Y))

    pred = (pred_prob_1 > threshold).astype(int)

    costs[(Y == 1) & (pred == 0)] = 1
    costs[(Y == 0) & (pred == 1)] = 5

    return costs

def bennett_ci(costs, delta=0.01):
    n = len(costs)
    mu = np.mean(costs)
    sigma2 = np.var(costs)
    M = 5

    term1 = np.sqrt((2 * sigma2 / n) * np.log(2/delta))
    term2 = (M / (3*n)) * np.log(2/delta)

    eps = term1 + term2
    return mu - eps, mu + eps


all_costs_at_optimal_threshold = all_costs(model, optimal_threshold, X_valid, Y_valid)


lower_bound_ben, upper_bound_ben = bennett_ci(all_costs_at_optimal_threshold, delta=0.01)


variance_of_C = np.var(all_costs_at_optimal_threshold)
interval_of_C = (lower_bound_ben, upper_bound_ben)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)